In [ ]:
#1. setup and loading 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#defining file path for dataset
file_path = '/Volumes/SHAYNE SSD/Project_Datasets/House_Price_Predictor/data_kaggle.csv'

#attempt to load data into dataframe 'df'
df = pd.read_csv(file_path)
# print("Dataset loaded succesfully. First 55 rows:")
# df.head()

In [ ]:
#2. Initial Data Exploration
print("Dataframe info:")
df.info() #tells us the number of entries, the column names, the number of non-null values, and the data type of each column.

print("\nDataframe shape:")
print(f"rows: {df.shape[0]} \n columns: {df.shape[1]}")

In [ ]:
#3. Data cleaning (price)
print("before clean:")
print(df["Price"].head())
print("\ndata type before clean:")
print(df["Price"].dtype)

df["Price_Original"] = df["Price"] #create copy of 'Price' for backup 
df["Price"] = df["Price"].str.replace("RM", "").str.replace(",", "") #replace "RM" and "," with blank (remove)

df["Price"] = pd.to_numeric(df["Price"], errors = 'coerce')

#check
print("\n\nafter clean:")
print(df["Price"].head())
print("\ndata type after clean:")
print(df["Price"].dtype)

In [ ]:
#4. Data Cleaning (size)
print("Before clean:")
print(df["Size"].head())
print("\nData type before clean:")
print(df["Size"].dtype)

df["Size_sqft"] = df["Size"].str.extract("(\d[\d,']*)")
df["Size_sqft"] = df["Size_sqft"].str.replace(",", "") #replace ","
df["Size_sqft"] = pd.to_numeric(df["Size_sqft"], errors = 'coerce') #convert str to int

#check
print("\n\nBefore clean:")
print(df["Size_sqft"].head())
print("\nAfter clean:")
print(df["Size_sqft"].dtype)

In [25]:
#handle missing data 
print("Before handle:")
print(df.isnull().sum())
print("\nTotal rows before drop")
print(df.shape[0])

df.dropna(subset = ["Price", "Size_sqft"], inplace = True) #subset argument tells it to only look for NaN in Price and Size_sqft

#check
print("\n\nAfter handle:")
print(df.isnull().sum())
print("\nTotal numbers after drop")
print(df.shape[0])

Before handle:
Location              0
Price                 0
Rooms              1591
Bathrooms          1885
Car Parks         17091
Property Type         0
Size                  0
Furnishing         6560
Price_Original        0
Size_sqft             0
dtype: int64

Total rows before drop
52523


After handle:
Location              0
Price                 0
Rooms              1591
Bathrooms          1885
Car Parks         17091
Property Type         0
Size                  0
Furnishing         6560
Price_Original        0
Size_sqft             0
dtype: int64

Total numbers after drop
52523
